In [1]:
import pandas as pd
import numpy as np
import random
import os
import pandas as pd
from datetime import datetime
import time

#Import Datasets

In [2]:
url = 'https://raw.githubusercontent.com/anhphuongnguyenquynh/session-based-recsys-fashion/main/dataset/item_features.csv'
item_features = pd.read_csv(url)

In [3]:
url = 'https://raw.githubusercontent.com/anhphuongnguyenquynh/session-based-recsys-fashion/main/dressipi_recsys2022_datasets.zip'
!wget $url
!unzip dressipi_recsys2022_datasets.zip

--2024-04-25 16:57:10--  https://raw.githubusercontent.com/anhphuongnguyenquynh/session-based-recsys-fashion/main/dressipi_recsys2022_datasets.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79384785 (76M) [application/zip]
Saving to: ‘dressipi_recsys2022_datasets.zip’

dressipi_recsys2022 100%[===================>]  75.71M   215MB/s    in 0.4s    

2024-04-25 16:57:10 (215 MB/s) - ‘dressipi_recsys2022_datasets.zip’ saved [79384785/79384785]

Archive:  dressipi_recsys2022_datasets.zip
   creating: dressipi_recsys2022_dataset/
  inflating: dressipi_recsys2022_dataset/README.txt  
  inflating: dressipi_recsys2022_dataset/candidate_items.csv  
  inflating: dressipi_recsys2022_dataset/item_features.csv  
  inflating: dressipi_recsys2022_dataset/test_final_pu

In [4]:
!ls

dressipi_recsys2022_dataset  dressipi_recsys2022_datasets.zip  sample_data


In [5]:
train_sessions = pd.read_csv('dressipi_recsys2022_dataset/train_sessions.csv')
train_purchases = pd.read_csv('dressipi_recsys2022_dataset/train_purchases.csv')
item_features = pd.read_csv('dressipi_recsys2022_dataset/item_features.csv')
candidate_items = pd.read_csv('dressipi_recsys2022_dataset/candidate_items.csv')
test_full_sessions = pd.read_csv('dressipi_recsys2022_dataset/test_full_sessions.csv')
test_full_purchases = pd.read_csv('dressipi_recsys2022_dataset/test_full_purchases.csv')

In [6]:
train_sessions.head()

,session_id,item_id,date
0,3,9655,2020-12-18 21:25:00.373
1,3,9655,2020-12-18 21:19:48.093
2,13,15654,2020-03-13 19:35:27.136
3,18,18316,2020-08-26 19:18:30.833
4,18,2507,2020-08-26 19:16:31.211


In [7]:
#convert date to timestamp
train_sessions['date'] = pd.to_datetime(train_sessions['date'], format='ISO8601')
train_sessions['timestamp'] = train_sessions['date'].map(datetime.timestamp)
train_sessions.head(5)

,session_id,item_id,date,timestamp
0,3,9655,2020-12-18 21:25:00.373,1.608327e+09
1,3,9655,2020-12-18 21:19:48.093,1.608326e+09
2,13,15654,2020-03-13 19:35:27.136,1.584128e+09
3,18,18316,2020-08-26 19:18:30.833,1.598470e+09
4,18,2507,2020-08-26 19:16:31.211,1.598469e+09


In [8]:
#convert date to timestamp
test_full_sessions['date'] = pd.to_datetime(test_full_sessions['date'], format='ISO8601')
test_full_sessions['timestamp'] = test_full_sessions['date'].map(datetime.timestamp)
test_full_sessions.head(5)

,session_id,item_id,date,timestamp
0,26,19185,2021-06-16 09:53:54.158,1.623837e+09
1,61,27088,2021-06-01 08:12:39.664,1.622535e+09
2,61,5581,2021-06-01 08:12:40.534,1.622535e+09
3,96,11693,2021-06-19 17:48:05.227,1.624125e+09
4,96,18298,2021-06-19 17:49:08.589,1.624125e+09


# Train/Test split
The dataset is splited already:
- train_sessions for training
- test_full_sessions for the last month for tesing

In [9]:
dataset = train_sessions
test = test_full_sessions

In [10]:
print(dataset.shape)
print(test.shape)

(4743820, 4)
(595992, 4)


In [11]:
dataset.head()

,session_id,item_id,date,timestamp
0,3,9655,2020-12-18 21:25:00.373,1.608327e+09
1,3,9655,2020-12-18 21:19:48.093,1.608326e+09
2,13,15654,2020-03-13 19:35:27.136,1.584128e+09
3,18,18316,2020-08-26 19:18:30.833,1.598470e+09
4,18,2507,2020-08-26 19:16:31.211,1.598469e+09


In [12]:
dataset['session_id']

0                3
1                3
2               13
3               18
4               18
            ...   
4743815    4440001
4743816    4440001
4743817    4440001
4743818    4440001
4743819    4440001
Name: session_id, Length: 4743820, dtype: int64

In [13]:

#Lấy ngẫu nhiên 1/4 số lượng các session cho huấn luyện
# list các sessionIds
import numpy as np
sessIds = list(dataset['session_id'].unique())
# Lấy ngẫu nhiên 1/4 số lượng các session
n_filter = int(len(sessIds)/4)
np.random.shuffle(sessIds)
sessIdsFilter = sessIds[:n_filter]
# Lựa chọn các 1/4 session làm dataset train (dữ liệu này bao gồm cả train và validation)
# Set index là sessionId để filter nhanh hơn
dataset.set_index('session_id', inplace=True)
dataset_filter = dataset[dataset.index.isin(sessIdsFilter)]
dataset_filter = dataset_filter.reset_index()
print(dataset_filter.shape)

(1180645, 4)


In [14]:
# Lấy ra dictionary có dạng {SessionId:{ItemId1:Timestamp1, ItemId2:Timestamp2, ...}}
train_sess = dataset_filter[['session_id', 'item_id', 'timestamp']].groupby('session_id').apply(lambda x: dict(zip(x['item_id'], x['timestamp'])))
test_sess = test[['session_id', 'item_id', 'timestamp']].groupby('session_id').apply(lambda x: dict(zip(x['item_id'], x['timestamp'])))

In [15]:
train_sess

session_id
48             {8398: 1586971062.594, 26404: 1586971719.697}
49                                   {21358: 1588336430.844}
154                                  {21152: 1587713021.545}
184                                  {14383: 1617500534.691}
195        {4086: 1609848055.97, 16626: 1609805948.996, 1...
                                 ...                        
4439891                              {16607: 1608153366.575}
4439894    {10186: 1617194216.962, 1103: 1617193787.576, ...
4439900    {5515: 1598559911.646, 19475: 1598558432.32, 5...
4439936                              {23764: 1618584742.939}
4439968    {26650: 1591897126.2, 22773: 1591896462.481, 1...
Length: 250000, dtype: object

#Preprocessing data

In [16]:
sessDict = {214834865: '1396808691.295000', 214706441: '1396808691.426000', 214820225: '1396808691.422000'}

def _preprocess_sess_dict(sessDict):
    sessDictTime = dict([(v, k) for (k, v) in sessDict.items()])
    sessSort = sorted(sessDictTime.items(), reverse = False)
    times = [item[0] for item in sessSort]
    itemIds = [item[1] for item in sessSort]
    inp_seq = []
    labels = []
    inp_time = []

    for i in range(len(sessSort)):
        if i >= 1:
            inp_seq += [itemIds[:i]]
            labels += [itemIds[i]]
            inp_time += [times[i]]
    return inp_seq, inp_time, labels, itemIds

inp_seq, inp_time, labels, itemIds = _preprocess_sess_dict(sessDict)
print('input sequences: ', inp_seq)
print('input times: ', inp_time)
print('targets: ', labels)
print('sequence: ', itemIds)

input sequences:  [[214834865], [214834865, 214820225]]
input times:  ['1396808691.422000', '1396808691.426000']
targets:  [214820225, 214706441]
sequence:  [214834865, 214820225, 214706441]


In [17]:
#Khởi tạo chuỗi input và output cho toàn bộ các session
def _preprocess_data(data_sess):
    inp_seqs = []
    inp_times = []
    labels = []
    sequences = []
    sessIds = list(data_sess.index)
    for sessId in sessIds:
        sessDict = data_sess.loc[sessId]
        inp_seq, inp_time, label, sequence = _preprocess_sess_dict(sessDict)
        inp_seqs += inp_seq
        inp_times += inp_time
        labels += label
        sequences += sequence
    return inp_seqs, inp_times, labels, sequences

train_inp_seqs, train_inp_dates, train_labs, train_sequences = _preprocess_data(train_sess)
test_inp_seqs, test_inp_dates, test_labs, test_sequences = _preprocess_data(test_sess)

train = (train_inp_seqs, train_labs)
test = (test_inp_seqs, test_labs)

print('Done.')

Done.


Lưu dữ liệu train/test

In [18]:
import pickle
import os

def _save_file(filename, obj):
  with open(filename, 'wb') as fn:
    pickle.dump(obj, fn)

# Tạo folder yoochoose-data-4 để lưu dữ liệu train/test nếu chưa tồn tại
if not os.path.exists('dressipi_data_train0.2'):
  os.mkdir('dressipi_data_train0.2')

# Lưu train/test
_save_file('dressipi_data_train0.2/train02.pkl', train)
_save_file('dressipi_data_train0.2/test02.pkl', test)

In [19]:
import pickle

def _load_file(filename):
  with open(filename, 'rb') as fn:
    data = pickle.load(fn)
  return data

# Load dữ liệu train/test từ folder
train = _load_file('dressipi_data_train0.2/train02.pkl')
test = _load_file('dressipi_data_train0.2/test02.pkl')

# Khởi tạo dictionary

In [20]:
# Các token default
PAD_token = 0  # token padding cho câu ngắn

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.item2index = {}
        self.item2count = {}
        self.index2item = {PAD_token: "PAD"}
        self.num_items = 1  # số lượng mặc định ban đầu là 1 ứng với PAD_token

    def addSenquence(self, data):
        for sequence in data:
          for item in sequence:
              self.addItem(item)

    # Thêm một item vào hệ thống
    def addItem(self, item):
        if item not in self.item2index:
            self.item2index[item] = self.num_items
            self.item2count[item] = 1
            self.index2item[self.num_items] = item
            self.num_items += 1
        else:
            self.item2count[item] += 1

    # Loại các item dưới ngưỡng xuất hiện min_count
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_items = []

        for k, v in self.item2count.items():
            if v >= min_count:
                keep_items.append(k)

        print('keep_items {} / {} = {:.4f}'.format(
            len(keep_items), len(self.item2index), len(keep_items) / len(self.item2index)
        ))

        # Khởi tạo lại từ điển
        self.item2index = {}
        self.item2count = {}
        self.index2item = {PAD_token: "PAD"}
        self.num_items = 1

        # Thêm các items vào từ điển
        for item in keep_items:
            self.addItem(item)

    # Hàm convert sequence về chuỗi các indices
    def _seqItem2seqIndex(self, x):
        return [voc.item2index[item] if item in voc.item2index else 0 for item in x]

In [21]:
#Lấy toàn bộ list các itemIds trong các session
from itertools import chain
seq_targets = [train[1]] + [test[1]]
sessionIds = list(chain.from_iterable(seq_targets))
sessionIds = set(sessionIds)
print('Number of sessionIds: ', len(sessionIds))

Number of sessionIds:  19822


In [22]:
#Khởi tạo vocabulary cho bộ dữ liệu
voc = Voc('DictItemId')
voc.addSenquence(seq_targets)

# Convert thử nghiệm một sequence itemIds
print('sequence of itemIds: ', train[0][7])
print('converted indices: ', voc._seqItem2seqIndex(train[0][7]))

sequence of itemIds:  [16626, 22694, 16075, 26835, 23638, 21018, 13717]
converted indices:  [8717, 2, 3, 4, 5, 6, 7]


In [23]:
#Chuyển dữ liệu train, test từ item sang indices của item
train_x_index = [voc._seqItem2seqIndex(seq) for seq in train[0]]
test_x_index = [voc._seqItem2seqIndex(seq) for seq in test[0]]
train_y_index = voc._seqItem2seqIndex(train[1])
test_y_index = voc._seqItem2seqIndex(test[1])
train_index = (train_x_index, train_y_index)
test_index = (test_x_index, test_y_index)

# Phân chia tập Train/Test/Validation
Từ 2 tập dữ liệu train_index và test_index ban đầu ta sẽ phân chia thành 3 tập train/test và validation như sau:

Mỗi tập dữ liệu bao gồm 2 phần: input bao gồm chuỗi các itemId liên tiếp, output là itemId tiếp theo trong được khách hàng click.

Dữ liệu validation được rút ra từ dữ liệu train set theo tỷ lệ được qui định tại valid_portion. Phần còn lại của train set được dữ làm tập train set mới.

Dữ liệu test được tính toàn trực tiếp từ tập test set.

Đối với dữ liệu input, chuỗi dữ liệu sẽ được truncate về độ dài maxlen nếu nó vượt qua maxlen.

In [24]:
def load_data(root='', valid_portion=0.1, maxlen=19, sort_by_len=False, train_set=None, test_set=None):
    """Load dataset từ root
    root: folder dữ liệu train, trong trường hợp train_set, test_set tồn tại thì không sử dụng train_set và test_set
    valid_portion: tỷ lệ phân chia dữ liệu validation/train
    maxlen: độ dài lớn nhất của sequence
    sort_by_len: có sort theo chiều dài các session trước khi chia hay không?
    train_set: training dataset
    test_set:  test dataset
    """

    # Load the dataset
    if train_set is None and test_set is None:
        path_train_data = os.path.join(root, 'train.pkl')
        path_test_data = os.path.join(root, 'test.pkl')
        with open(path_train_data, 'rb') as f1:
            train_set = pickle.load(f1)

        with open(path_test_data, 'rb') as f2:
            test_set = pickle.load(f2)

    if maxlen:
        new_train_set_x = []
        new_train_set_y = []
        # Lọc dữ liệu sequence đến maxlen
        for x, y in zip(train_set[0], train_set[1]):
            if len(x) < maxlen:
                new_train_set_x.append(x)
                new_train_set_y.append(y)
            else:
                new_train_set_x.append(x[:maxlen])
                new_train_set_y.append(y)
        train_set = (new_train_set_x, new_train_set_y)
        del new_train_set_x, new_train_set_y

        new_test_set_x = []
        new_test_set_y = []
        for xx, yy in zip(test_set[0], test_set[1]):
            if len(xx) < maxlen:
                new_test_set_x.append(xx)
                new_test_set_y.append(yy)
            else:
                new_test_set_x.append(xx[:maxlen])
                new_test_set_y.append(yy)
        test_set = (new_test_set_x, new_test_set_y)
        del new_test_set_x, new_test_set_y

    # phân chia tập train thành train và validation
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    (test_set_x, test_set_y) = test_set

    # Trả về indices thứ tự độ dài của mỗi phần tử trong seq
    def len_argsort(seq):
        return sorted(range(len(seq)), key=lambda x: len(seq[x]))

    # Sắp xếp session theo độ dài tăng dần
    if sort_by_len:
        sorted_index = len_argsort(test_set_x)
        test_set_x = [test_set_x[i] for i in sorted_index]
        test_set_y = [test_set_y[i] for i in sorted_index]

        sorted_index = len_argsort(valid_set_x)
        valid_set_x = [valid_set_x[i] for i in sorted_index]
        valid_set_y = [valid_set_y[i] for i in sorted_index]

    train = (train_set_x, train_set_y)
    valid = (valid_set_x, valid_set_y)
    test = (test_set_x, test_set_y)
    return train, valid, test

# Data Loader
Để streaming dữ liệu theo batch thì ta cần sử dụng class DataLoader của pytorch. Nó sẽ cho phép ta sử dụng generator để sinh dữ liệu cho từng batch huấn luyện. Do đó ta sẽ có thể huấn luyện được những mô hình từ dữ liệu có kích thước lớn hơn RAM gấp rất nhiều lần. Data Loader trong pytorch sẽ sử dụng dữ liệu là các class Dataset của pytorch như bên dưới

In [25]:
from torch.utils.data import Dataset

class RecSysDataset(Dataset):
    """define the pytorch Dataset class for yoochoose and diginetica datasets.
    """
    def __init__(self, data):
        self.data = data
        print('-'*50)
        print('Dataset info:')
        print('Number of sessions: {}'.format(len(data[0])))
        print('-'*50)

    def __getitem__(self, index):
        session_items = self.data[0][index]
        target_item = self.data[1][index]
        return session_items, target_item

    def __len__(self):
        return len(self.data[0])

In [26]:
#Hàm phụ trợ
import torch

def collate_fn(data):
    """
    Hàm số này sẽ được sử dụng để pad session về max length
    Args:
      data: batch truyền vào
    return:
      batch data đã được pad length có shape maxlen x batch_size
    """
    # Sort batch theo độ dài của input_sequence từ cao xuống thấp
    data.sort(key=lambda x: len(x[0]), reverse=True)
    lens = [len(sess) for sess, label in data]
    labels = []
    # Padding batch size
    padded_sesss = torch.zeros(len(data), max(lens)).long()
    for i, (sess, label) in enumerate(data):
        padded_sesss[i,:lens[i]] = torch.LongTensor(sess)
        labels.append(label)

    # Transpose dữ liệu từ batch_size x maxlen --> maxlen x batch_size
    padded_sesss = padded_sesss.transpose(0,1)
    return padded_sesss, torch.tensor(labels).long(), lens

#Metric

In [27]:
import torch

#indices (Bxk): torch.LongTensor. Top-k indices predicted from the model
#targets (B): torch.LongTensor. actual target indices

def get_recall(indices, targets):
    """
    Tính toán chỉ số recall cho một tập hợp predictions và targets
    Args:
        indices (Bxk): torch.LongTensor. top-k indices được dự báo từ mô hình model.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        recall (float): the recall score
    """
    # copy targets k lần để trở thành kích thước Bxk
    targets = targets.view(-1, 1).expand_as(indices)
    # print('targets', targets)
    # so sánh targets với indices để tìm ra vị trí mà khách hàng sẽ hit.
    hits = (targets == indices).to(device)
    # print('hits', hits)
    hits = hits.double()
    if targets.size(0) == 0:
        return 0
    # Đếm số hit
    n_hits = torch.sum(hits)
    # print('n_hits', n_hits)
    recall = n_hits / targets.size(0)
    # print('recall check', recall)
    # print(recall.dtype)
    return recall


def get_mrr(indices, targets):
    """
    Tính toán chỉ số MRR cho một tập hợp predictions và targets
    Args:
        indices (Bxk): torch.LongTensor. top-k indices được dự báo từ mô hình model.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        recall (float): the MRR score
    """
    tmp = targets.view(-1, 1)
    # print('tmp',tmp)
    targets = tmp.expand_as(indices)
    # print('targets', targets)
    hits = (targets == indices).to(device)
    # print('hits', hits)
    hits = hits.double()
    if hits.sum() == 0:
      return 0
    argsort = []
    for i in np.arange(hits.shape[0]):
      index_col = torch.where(hits[i, :] == 1)[0]+1
      if index_col.shape[0] != 0:
        argsort.append(index_col.double())
    inv_argsort = (1/item for item in argsort)
    mrr = sum(inv_argsort)/hits.shape[0]
    mrr_item = mrr.item()
    mrr = torch.tensor(mrr_item, dtype=torch.float64)
    return mrr

def get_hitrate(indices, targets):
    #inputs: indices and targets
    #returns: if just one item in targets hit in indices. return 1
    # copy targets k lần để trở thành kích thước Bxk
    targets = targets.view(-1, 1).expand_as(indices)
    # so sánh targets với indices để tìm ra vị trí mà khách hàng sẽ hit.
    hits = (targets == indices).to(device)
    hits = hits.double()
    if targets.size(0) == 0:
        return 0
    # Đếm số hit
    n_hits = torch.sum(hits)
    if n_hits == 0:
      hitrate = 0
    else:
      hitrate = 1
    #convert to torch
    hitrate = torch.tensor(hitrate, dtype=torch.float64)
    return hitrate


def evaluate(logits, targets, k=20):
    """
    Đánh giá model sử dụng Recall@K, MRR@K scores.
    Args:
        logits (B,C): torch.LongTensor. giá trị predicted logit cho itemId tiếp theo.
        targets (B): torch.LongTensor. actual target indices.
    Returns:
        recall (float): the recall score
        mrr (float): the mrr score
    """
    # Tìm ra indices của topk lớn nhất các giá trị dự báo.
    _, indices = torch.topk(logits, k, -1)
    # print('logits', logits)
    # print('indices', indices)
    recall = get_recall(indices, targets)
    mrr = get_mrr(indices, targets)
    hitrate = get_hitrate(indices, targets)
    # print(recall.dtype)
    # print(mrr.dtype)
    # print(hitrate.dtype)
    return recall, mrr, hitrate

In [28]:
import torch
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

logits = torch.tensor([[0.1, 0.2, 0.7],
                       [0.4, 0.1, 0.5],
                       [0.1, 0.2, 0.7]]).to(device)

targets = torch.tensor([1, 2, 2]).to(device)



In [29]:
evaluate(logits = logits, targets = targets, k = 2)

(tensor(1., dtype=torch.float64),
 tensor(0.8333, dtype=torch.float64),
 tensor(1., dtype=torch.float64))

In [30]:
recall, mrr, hitrate = evaluate(logits, targets, k = 2)
print(recall)
print('---'*5)
print(mrr)
print('---'*5)
print(hitrate)

tensor(1., dtype=torch.float64)
---------------
tensor(0.8333, dtype=torch.float64)
---------------
tensor(1., dtype=torch.float64)


In [31]:
print(recall.item())

1.0


In [32]:
x = torch.arange(11., 16.)
x
torch.topk(x, 3, -1)

torch.return_types.topk(
values=tensor([15., 14., 13.]),
indices=tensor([4, 3, 2]))

# Model NARM

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class NARM(nn.Module):
    def __init__(self, hidden_size, n_items, embedding_dim, n_layers=1, dropout=0.25):
        super(NARM, self).__init__()
        self.hidden_size = hidden_size
        self.n_items = n_items
        self.embedding_dim = embedding_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(self.n_items, self.embedding_dim, padding_idx = 0)
        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        # set bidirectional = True for bidirectional
        # https://pytorch.org/docs/stable/nn.html?highlight=gru#torch.nn.GRU to get more information
        self.gru = nn.GRU(input_size = hidden_size, # number of expected feature of input x
                          hidden_size = hidden_size, # number of expected feature of hidden state
                          num_layers = n_layers, # number of GRU layers
                          dropout=(0 if n_layers == 1 else dropout), # dropout probability apply in encoder network
                          bidirectional=True # one or two directions.
                         )
        self.emb_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(self.embedding_dim, self.hidden_size, self.n_layers)
        self.a_1 = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.a_2 = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.v_t = nn.Linear(self.hidden_size, 1, bias=False)
        self.ct_dropout = nn.Dropout(0.5)
        self.b = nn.Linear(self.embedding_dim, 2 * self.hidden_size, bias=False)
        self.sf = nn.Softmax()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def forward(self, input_seq, input_lengths, hidden=None):
        """
        input_seq: Batch input_sequence. Shape: max_len x batch_size
        input_lengths: Batch input lengths. Shape: batch_size
        """
        # Step 1: Convert sequence indexes to embeddings
        # shape: (max_length , batch_size , hidden_size)
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module. Padding zero when length less than max_length of input_lengths.
        # shape: (max_length , batch_size , hidden_size)
        packed = pack_padded_sequence(embedded, input_lengths)

        # Step 2: Forward packed through GRU
        # outputs is output of final GRU layer
        # hidden is concatenate of all hidden states corresponding with each time step.
        # outputs shape: (max_length , batch_size , hidden_size x num_directions)
        # hidden shape: (n_layers x num_directions , batch_size , hidden_size)
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding. Revert of pack_padded_sequence
        # outputs shape: (max_length , batch_size , hidden_size x num_directions)
        outputs, length = pad_packed_sequence(outputs)

        # Step 3: Global Encoder & Local Encoder
        # num_directions = 1 -->
        # outputs shape:(max_length , batch_size , hidden_size)
        # hidden shape: (n_layers , batch_size , hidden_size)
        # lấy hidden state tại time step cuối cùng
        ht = hidden[-1]
        # reshape outputs
        outputs = outputs.permute(1, 0, 2) # [batch_size, max_length, hidden_size]
        c_global = ht
        # Flatten outputs thành shape: [batch_size, max_length, hidden_size]
        gru_output_flatten = outputs.contiguous().view(-1, self.hidden_size)
        # Thực hiện một phép chiếu linear projection để tạo các latent variable có shape [batch_size, max_length, hidden_size]
        q1 = self.a_1(gru_output_flatten).view(outputs.size())
        # Thực hiện một phép chiếu linear projection để tạo các latent variable có shape [batch_size, max_length, hidden_size]
        q2 = self.a_2(ht)
        # Ma trận mask đánh dấu vị trí khác 0 trên padding sequence.
        mask = torch.where(input_seq.permute(1, 0) > 0, torch.tensor([1.], device = self.device), torch.tensor([0.], device = self.device)) # batch_size x max_len
        # Điều chỉnh shape
        q2_expand = q2.unsqueeze(1).expand_as(q1) # shape [batch_size, max_len, hidden_size]
        q2_masked = mask.unsqueeze(2).expand_as(q1) * q2_expand # batch_size x max_len x hidden_size
        # Tính trọng số alpha đo lường similarity giữa các hidden state
        alpha = self.v_t(torch.sigmoid(q1 + q2_masked).view(-1, self.hidden_size)).view(mask.size()) # batch_size x max_len
        alpha_exp = alpha.unsqueeze(2).expand_as(outputs) # batch_size x max_len x hidden_size
        # Tính linear combinition của các hidden state
        c_local = torch.sum(alpha_exp * outputs, 1) # (batch_size x hidden_size)

        # Véc tơ combinition tổng hợp
        c_t = torch.cat([c_local, c_global], 1) # batch_size x (2*hidden_size)
        c_t = self.ct_dropout(c_t)
        # Tính scores

        # Step 4: Decoder
        # embedding cho toàn bộ các item
        item_indices = torch.arange(self.n_items).to(device) # 1 x n_items
        item_embs = self.embedding(item_indices) # n_items x embedding_dim
        # reduce dimension by bi-linear projection
        B = self.b(item_embs).permute(1, 0) # (2*hidden_size) x n_items
        scores = torch.matmul(c_t, B) # batch_size x n_items
        # scores = self.sf(scores)
        return scores

In [34]:
#Kiểm tra model NARM
# Thử nghiệm model bằng cách giả lập 1 input và thực hiện quá trình feed forward
from torch import nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hidden_size = 3
n_layers = 7
# embedding = nn.Embedding(11000, hidden_size)
input_variable = torch.tensor([[  66,  369,   66, 1272],
                                [ 567,  183,   28,  616],
                                [ 392, 1558, 1143,  175],
                                [ 394,   31,   31, 5558],
                                [   0,    0,    0,    0]]).to(device)

lengths =  torch.tensor([5, 5, 5, 5]).to(device)
print('input_seq: \n', input_variable)
print('input_lengths: \n', lengths)
model_test = NARM(hidden_size = hidden_size, n_items  = 100000, embedding_dim = 100, n_layers=1, dropout=0.25).to(device)
print('model phrase: \n', model_test)
scores = model_test.forward(input_seq = input_variable, input_lengths = lengths)
print('probability distribution: ', scores.shape)

input_seq: 
 tensor([[  66,  369,   66, 1272],
        [ 567,  183,   28,  616],
        [ 392, 1558, 1143,  175],
        [ 394,   31,   31, 5558],
        [   0,    0,    0,    0]])
input_lengths: 
 tensor([5, 5, 5, 5])
model phrase: 
 NARM(
  (embedding): Embedding(100000, 100, padding_idx=0)
  (gru): GRU(100, 3)
  (emb_dropout): Dropout(p=0.25, inplace=False)
  (a_1): Linear(in_features=3, out_features=3, bias=False)
  (a_2): Linear(in_features=3, out_features=3, bias=False)
  (v_t): Linear(in_features=3, out_features=1, bias=False)
  (ct_dropout): Dropout(p=0.5, inplace=False)
  (b): Linear(in_features=100, out_features=6, bias=False)
  (sf): Softmax(dim=None)
)
probability distribution:  torch.Size([4, 100000])


#Validation

In [35]:
def validate(valid_loader, model):
    model.eval()
    recalls = []
    mrrs = []
    hitrates = []
    with torch.no_grad():
        for seq, target, lens in valid_loader:
            seq = seq.to(device)
            target = target.to(device)

            outputs = model(seq, lens)

            logits = F.softmax(outputs, dim = 1)
            recall, mrr, hitrate = evaluate(logits, target, k = args['topk'])
            recalls.append(recall)
            mrrs.append(mrr)
            hitrates.append(hitrate)

    mean_recall = torch.mean(torch.stack(recalls))
    # mean_mrr = torch.mean(torch.stack(mrrs))
    mean_mrr = torch.mean(torch.stack([torch.tensor(float(mrr)) for mrr in mrrs]))
    mean_hitrate = torch.mean(torch.stack(hitrates))
    return mean_recall, mean_mrr, mean_hitrate

#Training model

In [36]:
import os
import time
import random
import argparse
import pickle
import numpy as np
from tqdm import tqdm
from os.path import join

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torch.backends import cudnn

args = {
    'dataset_path':'../input/dressipi_recsys2022_dataset/train_sessions.csv',
    'batch_size': 256,
    'hidden_size': 100,
    'embed_dim': 50,
    'epoch': 5,
    'lr':0.001,
    'lr_dc':0.1,
    'lr_dc_step':80,
    'test':None,
    'topk':10,
    'valid_portion':0.1
}

here = os.path.dirname(os.getcwd())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def main():
    print('Loading data...')
    train_data, valid_data, test_data = load_data(train_set=train_index, test_set=test_index)
    train_data = RecSysDataset(train_data)
    valid_data = RecSysDataset(valid_data)
    test_data = RecSysDataset(test_data)
    train_loader = DataLoader(train_data, batch_size = args['batch_size'], shuffle = True, collate_fn = collate_fn)
    valid_loader = DataLoader(valid_data, batch_size = args['batch_size'], shuffle = False, collate_fn = collate_fn)
    test_loader = DataLoader(test_data, batch_size = args['batch_size'], shuffle = False, collate_fn = collate_fn)
    print('Complete load data!')
    n_items = voc.num_items
    model = NARM(hidden_size = args['hidden_size'], n_items = n_items, embedding_dim = args['embed_dim'], n_layers=2, dropout=0.25).to(device)
    print('complete load model!')

    if args['test'] == 'store_true':
        ckpt = torch.load('latest_checkpoint.pth.tar')
        model.load_state_dict(ckpt['state_dict'])
        recall, mrr, hitrate = validate(test_loader, model)
        print("Test: Recall@{}: {:.4f}, MRR@{}: {:.4f}, HitRate@{}: {:.4f}".format(args['topk'], recall, args['topk'], mrr, args['topk'], hitrate))
        return model

    optimizer = optim.Adam(model.parameters(), args['lr'])
    criterion = nn.CrossEntropyLoss()
    scheduler = StepLR(optimizer, step_size = args['lr_dc_step'], gamma = args['lr_dc'])

    print('start training!')
    for epoch in tqdm(range(args['epoch'])):
        # train for one epoch
        trainForEpoch(train_loader, model, optimizer, epoch, args['epoch'], criterion, log_aggr = 1000)
        scheduler.step(epoch = epoch)
        recall, mrr, hitrate = validate(valid_loader, model)
        print('Epoch {} validation: Recall@{}: {:.4f}; MRR@{}: {:.4f}; HitRate@{}: {:.4f}\n.  '.format(epoch, args['topk'], recall, args['topk'], mrr, args['topk'], hitrate, args['topk']))
        print('mrr', mrr)
        print('hitrate',hitrate)

        # store best loss and save a model checkpoint
        ckpt_dict = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }

        torch.save(ckpt_dict, 'latest_checkpoint.pth.tar')
    return model


def trainForEpoch(train_loader, model, optimizer, epoch, num_epochs, criterion, log_aggr=1000):
    model.train()

    sum_epoch_loss = 0

    start = time.time()
    for i, (seq, target, lens) in enumerate(train_loader):
        seq = seq.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        outputs = model(seq, lens)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        loss_val = loss.item()
        sum_epoch_loss += loss_val

        iter_num = epoch * len(train_loader) + i + 1

        if i % log_aggr == 0:
            print('[TRAIN] epoch %d/%d  observation %d/%d batch loss: %.4f (avg %.4f) (%.2f im/s)'
                % (epoch + 1, num_epochs, i, len(train_loader), loss_val, sum_epoch_loss / (i + 1),
                  len(seq) / (time.time() - start)))

        start = time.time()

model = main()

Loading data...
--------------------------------------------------
Dataset info:
Number of sessions: 695455
--------------------------------------------------
--------------------------------------------------
Dataset info:
Number of sessions: 77273
--------------------------------------------------
--------------------------------------------------
Dataset info:
Number of sessions: 375638
--------------------------------------------------
Complete load data!
complete load model!
start training!


  0%|          | 0/5 [00:00<?, ?it/s]

[TRAIN] epoch 1/5  observation 0/2717 batch loss: 11.1510 (avg 11.1510) (63.51 im/s)
[TRAIN] epoch 1/5  observation 1000/2717 batch loss: 9.5489 (avg 9.7968) (205.41 im/s)
[TRAIN] epoch 1/5  observation 2000/2717 batch loss: 9.2343 (avg 9.5823) (214.00 im/s)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
 20%|██        | 1/5 [04:42<18:50, 282.66s/it]

Epoch 0 validation: Recall@10: 0.0123; MRR@10: 0.0046; HitRate@10: 0.9536
.  
mrr tensor(0.0046)
hitrate tensor(0.9536, dtype=torch.float64)
[TRAIN] epoch 2/5  observation 0/2717 batch loss: 9.1101 (avg 9.1101) (132.50 im/s)
[TRAIN] epoch 2/5  observation 1000/2717 batch loss: 9.1205 (avg 9.1672) (216.62 im/s)
[TRAIN] epoch 2/5  observation 2000/2717 batch loss: 9.0373 (avg 9.1453) (218.53 im/s)


...

Epoch 2 validation: Recall@10: 0.0236; MRR@10: 0.0086; HitRate@10: 0.9934
.  
mrr tensor(0.0086)
hitrate tensor(0.9934, dtype=torch.float64)
[TRAIN] epoch 4/5  observation 0/2717 batch loss: 8.7457 (avg 8.7457) (129.78 im/s)
[TRAIN] epoch 4/5  observation 1000/2717 batch loss: 8.6548 (avg 8.6121) (131.06 im/s)
[TRAIN] epoch 4/5  observation 2000/2717 batch loss: 8.5326 (avg 8.5858) (195.38 im/s)


 80%|████████  | 4/5 [18:36<04:38, 278.88s/it]

Epoch 3 validation: Recall@10: 0.0298; MRR@10: 0.0107; HitRate@10: 1.0000
.  
mrr tensor(0.0107)
hitrate tensor(1., dtype=torch.float64)
[TRAIN] epoch 5/5  observation 0/2717 batch loss: 8.2836 (avg 8.2836) (143.03 im/s)
[TRAIN] epoch 5/5  observation 1000/2717 batch loss: 8.3925 (avg 8.4325) (215.25 im/s)
[TRAIN] epoch 5/5  observation 2000/2717 batch loss: 8.2841 (avg 8.4065) (217.97 im/s)


100%|██████████| 5/5 [23:16<00:00, 279.25s/it]

Epoch 4 validation: Recall@10: 0.0353; MRR@10: 0.0122; HitRate@10: 1.0000
.  
mrr tensor(0.0122)
hitrate tensor(1., dtype=torch.float64)


In [37]:
os.listdir()
# chứa 'latest_checkpoint.pth.tar'

['.config',
 'dressipi_recsys2022_dataset',
 'latest_checkpoint.pth.tar',
 'dressipi_recsys2022_datasets.zip',
 'dressipi_data_train0.2',
 'sample_data']

In [39]:
import torch
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
PATH = 'latest_checkpoint.pth.tar'
model = NARM(hidden_size = args['hidden_size'], n_items = 19823, embedding_dim = args['embed_dim'], n_layers=2, dropout=0.25).to(device)
optimizer = optim.Adam(params = model.parameters(), lr=0.001)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])
epoch = checkpoint['epoch']

model.eval()

NARM(
  (embedding): Embedding(19823, 50, padding_idx=0)
  (gru): GRU(50, 100, num_layers=2)
  (emb_dropout): Dropout(p=0.25, inplace=False)
  (a_1): Linear(in_features=100, out_features=100, bias=False)
  (a_2): Linear(in_features=100, out_features=100, bias=False)
  (v_t): Linear(in_features=100, out_features=1, bias=False)
  (ct_dropout): Dropout(p=0.5, inplace=False)
  (b): Linear(in_features=50, out_features=200, bias=False)
  (sf): Softmax(dim=None)
)

In [40]:
# Lựa chọn ngẫu nhiên một session trên test
import numpy as np
i = np.random.randint(0, len(test_index[0]))
x = [test_index[0][i]]
y = [test_index[1][i]]
print('item indexes sequence input: ', x)
print('item index next output: ', y)

item indexes sequence input:  [[14384]]
item index next output:  [5019]


In [41]:
# Step 1: Khởi tạo test_loader để biến đổi dữ liệu session đưa vào mô hình
test_data = RecSysDataset([x, y])
test_loader = DataLoader(test_data, batch_size = args['batch_size'], shuffle = False, collate_fn = collate_fn)

# Step 2: Dự báo các indice tiếp theo mà khách hàng có khả năng click
def _preddict(loader, model):
    model.eval()
    recalls = []
    mrrs = []
    j = 1
    with torch.no_grad():
      for seq, target, lens in loader:
        seq = seq.to(device)
        target = target.to(device)
        outputs = model(seq, lens)
        logits = F.softmax(outputs, dim = 1)
        _, indices = torch.topk(logits, 20, -1)
        print('Is next clicked item in top 20 suggestions: ', (target in indices))
        print('Top 20 next item indices suggested: ')
    return indices

_preddict(test_loader, model)

--------------------------------------------------
Dataset info:
Number of sessions: 1
--------------------------------------------------
Is next clicked item in top 20 suggestions:  False
Top 20 next item indices suggested: 


tensor([[3475, 2089, 2279, 3224, 2153, 2317,  533, 6602, 1203,  717,  330, 2352,
          972, 4921, 1579,  430, 4532,  855, 4061,  520]])